In [ ]:
#Set up project root
project_root = sys.path.append(os.path.abspath(".."))

#import libraries
import yfinance as yf
import time
import pandas as pd
import random
from datetime import datetime
import os
import sys
from utils.file_utils import retrieve_single_company_info, date_today_string


#Dataframe with ticker, co_name, business_summary, industry/sector breakdown.. 
ticker_list_path = os.path.join("..","data","ticker_list",'ASX_tickers_final.csv')
ticker_list = pd.read_csv(ticker_list_path)

#Select the first 1666 tickers from ticker list
tickers = ticker_list['Ticker'][0:10]

#Process 20 tickers in each iteration. 
batch_size = 20

#for storing cross sectional stock data
# ticker_vals = {}
all_company_info = []

date_today = date_today_string()

#Loop through all tickers with step of batch size. 
for i in range(0, len(tickers), batch_size):
    #The last batch will have less than batch_size tickers.
    # Note python behavior: slicing a list by a number larger than the original list will not cause an out of index error. 
    # i.e.: if a list has 10 elements indexed 0-10, slicing it by ls[0:15] will return the 10 elements and not cause an error
    batch = tickers[i:i + batch_size]

    for ticker in batch:

        #fetches data for a particular ticker. Results are passed into different functions to avoid excessive number of API calls. 
        company = yf.Ticker(ticker)
        
        # Get company info
        retrieve_single_company_info(all_company_info, company, ticker)
      
        # Get financial statements
        income_statement = company.financials.T
        balance_sheet = company.balance_sheet.T
        cashflow_statement = company.cashflow.T

        # Display the data
        print("Income Statement:")
        print(income_statement)

        print("\nBalance Sheet:")
        print(balance_sheet)

        print("\nCash Flow Statement:")
        print(cashflow_statement)

        directory_date = f"../data/AU/financial_statements/{date_today}/"
        
        if not os.path.exists(directory_date):
            os.makedirs(directory_date)
            print(f"Directory '{directory_date}' created.")
        else:
            print(f"Directory '{directory_date}' already exists.")
        
        
        financial_statements_dir = os.path.join("..", "data", "AU", "financial_statements", date_today)

        # Save the data in csv files for future reference. Rewrite code to store in database in future. 
        csv_filename_IS = os.path.join(financial_statements_dir, f"{ticker}_IS.csv")
        income_statement.to_csv(csv_filename_IS)
        csv_filename_BS = os.path.join(financial_statements_dir, f"{ticker}_BS.csv")
        balance_sheet.to_csv(csv_filename_BS)
        csv_filename_CF = os.path.join(financial_statements_dir, f"{ticker}_CF.csv")
        cashflow_statement.to_csv(csv_filename_CF)

        print(f"Saved data")   

    time.sleep(10+random.random()*2)



Income Statement:
           Tax Effect Of Unusual Items Tax Rate For Calcs Total Unusual Items  \
2024-06-30                -137904000.0              0.312        -442000000.0   
2023-06-30                  17751000.0              0.291          61000000.0   
2022-06-30                 122010000.0              0.294         415000000.0   
2021-06-30            -71638949.494949           0.285414        -251000000.0   
2020-06-30                         NaN                NaN                 NaN   

           Total Unusual Items Excluding Goodwill  \
2024-06-30                           -442000000.0   
2023-06-30                             61000000.0   
2022-06-30                            415000000.0   
2021-06-30                           -251000000.0   
2020-06-30                                    NaN   

           Net Income From Continuing Operation Net Minority Interest  \
2024-06-30                                       9481000000.0           
2023-06-30                    

In [13]:
new_df = pd.DataFrame.from_dict(all_company_info)
new_df.set_index('ticker', inplace=True)
new_df = new_df.reset_index()
company_info = os.path.join("..", "data", "AU", "company_info","company_info.csv")
new_df.to_csv(company_info)


#Load Data using CSVs (Income statements, Balance sheets and CF statements) that have been downloaded previously. 


In [14]:
#Memory efficient implementation of load data frames
""" 
    Takes in tickers as inputs, gradually yields dataframes. 
    Concat all data frames using lazy loading. 
"""

#Set up project root
project_root = sys.path.append(os.path.abspath(".."))

#takes in tickers as inputs, gradually passes data frames. 
#uses a generator ins
import pandas as pd
import numpy as np
import os
from utils.file_utils import load_fin_data_yield, date_today_string
from utils.financial_formula_utils import create_prior_year_BS_item, compute_prior_year_BS_item_and_average, compute_rate_of_change, compute_ratio, add_rate_of_change_to_df, add_financial_ratio_to_df, retrieve_metric_3Y_time_series, retrieve_metrics_3Y_time_series
from src.constants import roc_is_metrics, roc_bs_metrics, financial_ratio_tuples

ticker_list_path = os.path.join("..","data","ticker_list",'ASX_tickers_final.csv')
ticker_list = pd.read_csv(ticker_list_path)[:1666]

tickers = ticker_list['Ticker']

directory_statements = os.path.join("..", "data", "AU", "financial_statements")


# date_today = date_today_string()
date_today = "20250312"

df_IS = pd.concat(load_fin_data_yield(tickers, "IS",date_today,directory_statements), axis=0)
df_BS = pd.concat(load_fin_data_yield(tickers, "BS",date_today,directory_statements), axis=0)
df_CF = pd.concat(load_fin_data_yield(tickers, "CF",date_today,directory_statements), axis=0)


In [15]:
#Merges Income Statement, Balance Sheet and Company Information
is_bs_co_df = df_IS.merge(df_BS, on=['ticker','FY_End'], how='outer')
is_bs_co_df = is_bs_co_df.sort_values(by = ['ticker','FY_End'])
is_bs_co_df = is_bs_co_df.merge(ticker_list,left_on='ticker', right_on="Ticker",how='inner')

#Create a new column FY based on FY_End date
is_bs_co_df["FY"] = is_bs_co_df["FY_End"].apply(lambda x:(x[:4]))

print(df_IS.shape)
print(df_BS.shape)
print(ticker_list.shape)
print(is_bs_co_df.shape)


(7153, 82)
(7282, 139)
(1666, 9)
(7368, 229)


In [16]:
roc_metrics = roc_is_metrics + roc_bs_metrics

is_bs_co_df = add_rate_of_change_to_df(is_bs_co_df, roc_metrics)
is_bs_co_df = add_financial_ratio_to_df(is_bs_co_df, financial_ratio_tuples)
is_bs_co_df

,ticker,FY_End,Tax Effect Of Unusual Items,Tax Rate For Calcs,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Net Interest Income,Interest Expense,...,ROIC,Net Income Margin,Operating Profit Margin,EBITDA Margin,EBIT Margin,Gross Profit Margin,Average Interest Rate,Cash/Assets Ratio,Total Debt/Assets ratio,Net Debt/Assets ratio
0,14D.AX,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14D.AX,2021-06-30,0.000000,0.000000,-2.933040e+06,-2.933040e+06,-5.974178e+06,136772.0000,-85421.0000,104249.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.113643,NaN
2,14D.AX,2022-06-30,0.000000,0.000000,-9.975160e+05,-9.975160e+05,-1.369310e+06,380879.0000,-257996.0000,276997.0000,...,-0.124460,-3.502887,-9.117608,-1.819948,-2.794290,1.000000,0.275976,0.275053,0.036456,NaN
3,14D.AX,2023-06-30,0.000000,0.000000,NaN,NaN,-1.831251e+06,30200.0000,28268.0000,1022.0000,...,-0.189670,NaN,NaN,NaN,NaN,NaN,0.003076,0.179424,0.023373,NaN
4,14D.AX,2024-06-30,0.000000,0.000000,6.982000e+03,6.982000e+03,-2.505500e+06,35242.0000,-10703.0000,40529.0000,...,-0.306851,NaN,NaN,NaN,NaN,NaN,0.074905,0.178600,0.091012,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7363,ZNO.AX,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7364,ZNO.AX,2021-06-30,0.000000,0.214433,NaN,NaN,4.404346e+06,334880.4094,-53272.2196,70005.5840,...,NaN,0.174414,0.210121,0.233401,0.220139,0.593434,NaN,NaN,0.065450,NaN
7365,ZNO.AX,2022-06-30,-280545.725943,0.048651,-5.766470e+06,-5.766470e+06,-1.027659e+07,279605.8802,-62794.9134,1617.0635,...,-0.692276,-1.263958,-0.714236,-1.374669,-1.409059,0.561394,0.001103,0.139966,0.056766,NaN
7366,ZNO.AX,2023-06-30,0.000000,0.280000,NaN,NaN,-3.219682e+05,394694.4776,-53960.3051,58503.1701,...,-0.031980,-0.105017,-1.755634,-0.059240,-0.187978,0.512577,0.062551,0.048926,0.064854,NaN


In [17]:
#Show metrics for one company. 
ticker_to_show = "GMG.AX"

financial_ratios_to_show = [i[2] for i in financial_ratio_tuples]
roc_to_show = ["Total Revenue", "Gross Profit", "Net Income Common Stockholders", "EBITDA", "EBIT","Interest Expense","Total Debt","Net Debt","Diluted EPS","Basic EPS"]
is_bs_co_df.loc[is_bs_co_df['ticker'] == ticker_to_show, ["ticker","FY_End"]+financial_ratios_to_show + [metric+ " y/y change" for metric in roc_to_show]] 

,ticker,FY_End,Total Asset Turnover,ROE,ROA,ROIC,Net Income Margin,Operating Profit Margin,EBITDA Margin,EBIT Margin,...,Total Revenue y/y change,Gross Profit y/y change,Net Income Common Stockholders y/y change,EBITDA y/y change,EBIT y/y change,Interest Expense y/y change,Total Debt y/y change,Net Debt y/y change,Diluted EPS y/y change,Basic EPS y/y change
3026,GMG.AX,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3027,GMG.AX,2021-06-30,NaN,NaN,NaN,NaN,1.245636,0.202101,1.326724,1.314332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3028,GMG.AX,2022-06-30,0.106921,0.230782,0.178280,0.198035,1.667399,0.407766,1.946911,1.938559,...,0.103179,0.518264,0.476707,0.618868,0.627121,10.912371,0.347445,0.558207,0.464373,0.460925
3029,GMG.AX,2023-06-30,0.085381,0.090561,0.068628,0.076890,0.803782,0.323131,1.039058,1.030453,...,-0.052161,-0.107341,-0.543087,-0.494142,-0.496171,0.115102,0.157710,0.088166,-0.545302,-0.546943
3030,GMG.AX,2024-06-30,0.080334,-0.005562,-0.004133,-0.004649,-0.051451,0.241650,0.011861,0.002757,...,-0.009533,0.048990,-1.063402,-0.988693,-0.997350,-0.758246,0.107838,-0.016246,-1.063961,-1.062651


In [18]:
#Takes in metric of a company and returns metric for the past three years
retrieve_metric_3Y_time_series(is_bs_co_df,"ROA")

,ROA_2022,ROA_2023,ROA_2024
ticker,,,
14D.AX,-0.106111,-0.168631,-0.262091
1AD.AX,-0.615396,-0.527830,-0.808843
1AE.AX,NaN,-0.970428,-0.719021
1AI.AX,-0.661666,-0.496735,-0.566012
1CG.AX,-3.008938,-0.734384,NaN
...,...,...,...
ZLD.AX,-0.309997,-0.155688,-1.871771
ZMI.AX,-0.061694,-0.050235,-0.171987
ZMM.AX,-0.527962,-0.759696,-0.734884


In [20]:
directory_date_metrics_over_time = os.path.join("..", "reports", "AU")

ls_metrics_to_show = ["ROE","EBIT Margin","ROA","ROIC"] 

df_comp = retrieve_metrics_3Y_time_series(is_bs_co_df,ls_metrics_to_show, ticker_list, directory_date_metrics_over_time)
df_comp


Directory '..\reports\AU' already exists.


,Ticker,sector,ROE_2022,ROE_2023,ROE_2024,EBIT Margin_2022,EBIT Margin_2023,EBIT Margin_2024,ROA_2022,ROA_2023,ROA_2024,ROIC_2022,ROIC_2023,ROIC_2024
1650,PNT.AX,Basic Materials,-0.189989,-0.191141,NaN,NaN,NaN,NaN,-0.182756,-0.183304,NaN,-0.189989,-0.191141,NaN
1082,ESR.AX,Basic Materials,-0.135169,-0.076960,-0.139937,NaN,-3.912145,NaN,-0.124565,-0.072181,-0.134626,-0.134073,-0.075080,-0.136600
853,ETM.AX,Basic Materials,-0.221808,-0.264664,NaN,NaN,NaN,NaN,-0.200452,-0.236600,NaN,-0.221808,-0.264664,NaN
714,EUR.AX,Basic Materials,-0.203432,-0.168568,-5.867545,-400.624365,NaN,NaN,-0.199388,-0.161688,-2.341511,-0.202833,-0.168568,-5.718612
1322,EV1.AX,Basic Materials,NaN,-1.428663,-1.319292,NaN,NaN,NaN,NaN,-1.322864,-1.166976,NaN,-1.428663,-1.319292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,KPO.AX,Utilities,-0.403538,-0.406249,-0.451752,NaN,NaN,NaN,-1.384394,-1.463851,-6.101089,-0.403538,-0.406249,-0.451752
409,LGI.AX,Utilities,NaN,0.150313,0.130615,NaN,0.268491,0.299585,NaN,0.095827,0.087829,NaN,0.120847,0.108669
51,APA.AX,Utilities,0.110989,0.158743,0.426051,0.340835,0.335797,0.560066,0.015080,0.016592,0.055209,0.018457,0.020434,0.067365
1583,H2G.AX,Utilities,-0.462544,-0.451156,NaN,-8.506716,-12.463018,NaN,-0.312779,-0.406081,NaN,-0.419745,-0.445466,NaN


In [21]:
#Option 1
import gradio as gr
import pandas as pd


# roc_to_show = ["Total Revenue","EBITDA"]
# roc_cols = [col for col in merge_is_df.columns if "y/y change" in col.lower()]
financial_ratios_to_show = [i[2] for i in financial_ratio_tuples]
ls_metrics_to_show = financial_ratios_to_show 

def show_dataframe(ticker1, ticker2, metric):
    ls_tickers = []
    ls_tickers.append(ticker1)
    ls_tickers.append(ticker2)

    df = is_bs_co_df[is_bs_co_df["ticker"].isin(ls_tickers)]

    df = retrieve_metric_3Y_time_series(df,metric)
    
    desired_cols = df.columns[:3]

    df = df.merge(ticker_list,left_index=True,right_on="Ticker",how="inner")

    df.sort_values(by="sector",inplace=True)
    df = df[["Ticker", "sector"] + list(desired_cols)]
    
    return gr.update(value = df , visible= True)


def show_industry_metrics_median(metric):
    
    df_ind = is_bs_co_df.groupby(by=["sector","FY"])[metric].median().unstack().reset_index()
    df_ind.rename_axis(None,axis = 1, inplace=True)
    df_ind = df_ind.replace([np.inf, -np.inf], np.nan)
    # col_df = df_ind.columns[:1] + df_ind.columns[-3:]
    # col_df =  df_ind.columns[-3:]
    # df_ind = df_ind[col_df]


    return gr.update(value = df_ind, visible=True)


with gr.Blocks() as demo:
    gr.Markdown("# ASX Stock Screener")
    with gr.Column(scale = 1):
        dropdown1 = gr.Dropdown(choices = list(ticker_list["Ticker"]), value="CBA.AX", label="Select Ticker 1")
        dropdown2 = gr.Dropdown(choices = list(ticker_list["Ticker"]), value="BHP.AX", label="Select Ticker 2")
        dropdown_m1 = gr.Dropdown(choices = ls_metrics_to_show, label="Metric to show 1")
        dropdown_m2 = gr.Dropdown(choices = ls_metrics_to_show, value = "ROA", label="Metric to show 2")
        

    with gr.Column(scale = 1):
        output1 = gr.DataFrame(visible = False, max_height=1000)
        output2 = gr.DataFrame(visible = False, max_height=1000)

    with gr.Column(scale = 1):
        
        dropdown_ind = gr.Dropdown(choices = ls_metrics_to_show, value="ROE", label="Select Metric for Industry")
        output_ind = gr.DataFrame(visible = False, max_height=1000)
        
    
    
    dropdown1.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m1], outputs=output1)
    dropdown1.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m2], outputs=output2)

    dropdown2.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m1], outputs=output1)
    dropdown2.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m1], outputs=output2)

    dropdown_m1.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m1], outputs=output1)
    dropdown_m2.change(show_dataframe, inputs=[dropdown1, dropdown2,dropdown_m2], outputs=output2)

    dropdown_ind.change(show_industry_metrics_median, inputs=[dropdown_ind], outputs=output_ind)

demo.launch()
    

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [22]:
def get_metric_industry_median(df, metric:str):
    df = df.groupby(by=["sector","FY"])[metric].median().unstack().reset_index()
    df.rename_axis(None,axis = 1, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    # df = df.iloc[:,-3:]
    return df

get_metric_industry_median(is_bs_co_df,"ROE")

,sector,2020,2021,2022,2023,2024
0,Basic Materials,NaN,-0.147033,-0.158312,-0.184986,-0.204501
1,Communication Services,NaN,-0.028903,-0.044114,-0.051473,-0.009390
2,Consumer Cyclical,NaN,0.203851,0.116939,0.075723,0.108387
3,Consumer Defensive,NaN,-0.290304,-0.031299,-0.045992,-0.016830
4,Energy,NaN,-0.143559,-0.094468,-0.109621,-0.087443
5,Financial,NaN,0.207565,0.080652,0.077339,0.084214
6,Financial Services,NaN,0.119039,0.055478,0.070106,0.074190
7,Healthcare,NaN,-0.555320,-0.375671,-0.375877,-0.389483
8,Industrials,NaN,-0.586473,0.056908,0.062976,0.072149
9,N.A,NaN,NaN,-1.220066,-0.815600,-0.239062
